In [36]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [37]:
df = pd.read_excel('hist_vacancy_rates.xlsx', header=3, engine='openpyxl')
df.head()

,Year and Quarter,United States,Inside MSAs,Inside Principal Cities1,Outside Principal Cities1 (Suburbs),Outside MSAs,Northeast,Midwest2,South,West
0,19563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1st…………......,6.2,4.8,NaN,NaN,8.9,3.3,5.2,7.9,9.7
2,2nd……….......,5.9,4.8,NaN,NaN,8.1,2.7,5.3,8.1,9.3
3,3rd………….......,6.3,5.1,NaN,NaN,8.3,3.3,6.3,8.5,8.3
4,4th…………......,5.8,4.6,NaN,NaN,8.0,3.1,5.6,8.1,7.4


In [38]:
df = df.iloc[:435]

In [39]:
df = df.dropna(subset='Year and Quarter')

In [40]:
curr_year = None
df['Year'] = 0
for idx, row in df.iterrows():
    if not str(row['Year and Quarter'])[:3] in ["1st", "2nd", "3rd", "4th"]:
        curr_year = str(row['Year and Quarter'])[:4]
        df.loc[idx, 'Year and Quarter'] = None
    else:
        df.loc[idx, 'Year'] = curr_year
        quarter = row['Year and Quarter'][:3]
        if quarter == "1st":
            quarter = 1
        elif quarter == "2nd":
            quarter = 2
        elif quarter == "3rd":
            quarter = 3
        elif quarter == "4th":
            quarter = 4
        else:
            quarter = None
        df.loc[idx, 'Year and Quarter'] = quarter

df = df.dropna(subset='Year and Quarter')
df.rename(columns={'Year and Quarter': 'Quarter'}, inplace=True)
year_col = df.pop('Year')
df.insert(0, 'Year', year_col)
df['Year'] = df['Year'].astype(int)
df['Quarter'] = df['Quarter'].astype(int)

C:\Users\Jeffrey\AppData\Local\Temp\ipykernel_9736\1579419041.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1956' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[idx, 'Year'] = curr_year


In [41]:
df = df.drop_duplicates(subset = ['Year', 'Quarter'], keep='first')
df['Month'] = df['Quarter'].map({1: 1, 2: 4, 3: 7, 4: 10})
df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(day=1))
df.set_index('Date', inplace=True)

In [42]:
df.head()

,Year,Quarter,United States,Inside MSAs,Inside Principal Cities1,Outside Principal Cities1 (Suburbs),Outside MSAs,Northeast,Midwest2,South,West,Month
Date,,,,,,,,,,,,
1956-01-01,1956,1,6.2,4.8,NaN,NaN,8.9,3.3,5.2,7.9,9.7,1
1956-04-01,1956,2,5.9,4.8,NaN,NaN,8.1,2.7,5.3,8.1,9.3,4
1956-07-01,1956,3,6.3,5.1,NaN,NaN,8.3,3.3,6.3,8.5,8.3,7
1956-10-01,1956,4,5.8,4.6,NaN,NaN,8.0,3.1,5.6,8.1,7.4,10
1957-01-01,1957,1,5.3,4.2,NaN,NaN,7.1,3.5,5.0,6.4,6.9,1


In [43]:
df.dtypes

Year                                     int32
Quarter                                  int32
United States                          float64
Inside MSAs                            float64
Inside Principal Cities1               float64
Outside Principal Cities1 (Suburbs)    float64
Outside MSAs                           float64
Northeast                              float64
Midwest2                               float64
South                                  float64
West                                   float64
Month                                    int64
dtype: object

In [44]:
df.to_csv('clean_vacancy_data.csv', index=True)